In [15]:
import numpy as np
import itertools
from numpy.linalg import matrix_rank
from scipy.linalg import null_space, lstsq
from src.data.synthetic_data import SyntheticDataGenerator

# Example parition
# # Species = [S1, S2, S3, S4, S5, S6, S7, S8, S9, S_out]
# # Inputs = [S1, S2, S5, S8]
# # Hidden = [S3, S4, S6, S7, S9]
# # Output = S_out
# All that matters in the number in each group, not the specific species labels which are equivalent up to change of names.

# Example reaction
# # S_0 + S_1 <=> S_2 + S_3
# # S_3 + S_4 <=> S_5 + S_6
# # S_6 + S_7 <=> S_8 + S_out

generator = SyntheticDataGenerator(
    number_of_nonoutput_species=9, 
    reaction_rate_ranges={'k_f': (0, 1.0), 'k_r': (0, 1.0)},
    initial_concentration_range = (1, 10)
    )  


In [9]:
partition = generator.partitions[3]
partition


Partition(inputs=['S_1', 'S_2', 'S_3', 'S_4'], hidden=['S_5', 'S_6', 'S_7', 'S_8', 'S_9'], output='S_out')

In [10]:
#%%timeit
seed = 50
atomic_components = 7
max_components_per_species = 3
rank_target = None
print(f"Original partition: {partition}")
for _ in range(1):
    A = generator.sample_composition_matrix(partition, atomic_components, max_components_per_species, rank_target, seed)
    display(A)
    display(generator.structure_metadata(A, partition))
    reactions = generator.generate_candidate_reactions(partition, A)
    display(reactions)
    print(f"Pruned partition:")
    display(generator.prune_unused_species(A, partition, reactions))


Original partition: Partition(inputs=['S_1', 'S_2', 'S_3', 'S_4'], hidden=['S_5', 'S_6', 'S_7', 'S_8', 'S_9'], output='S_out')


array([[0, 1, 0, 2, 3, 1, 0, 1, 1, 0],
       [1, 0, 2, 0, 0, 0, 3, 2, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0, 2, 2],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

{'S_1': {'composition': array([0, 1, 1, 1, 0, 0, 0]), 'type': 'input'},
 'S_2': {'composition': array([1, 0, 1, 1, 0, 0, 0]), 'type': 'input'},
 'S_3': {'composition': array([0, 2, 1, 0, 0, 0, 0]), 'type': 'input'},
 'S_4': {'composition': array([2, 0, 0, 1, 0, 0, 0]), 'type': 'input'},
 'S_5': {'composition': array([3, 0, 0, 0, 0, 0, 0]), 'type': 'hidden'},
 'S_6': {'composition': array([1, 0, 1, 0, 1, 0, 0]), 'type': 'hidden'},
 'S_7': {'composition': array([0, 3, 0, 0, 0, 0, 0]), 'type': 'hidden'},
 'S_8': {'composition': array([1, 2, 0, 0, 0, 0, 0]), 'type': 'hidden'},
 'S_9': {'composition': array([1, 0, 2, 0, 0, 0, 0]), 'type': 'hidden'},
 'S_out': {'composition': array([0, 0, 2, 0, 1, 0, 0]), 'type': 'output'}}

[Reaction(reactants=(2, 3), products=(1, 7)),
 Reaction(reactants=(7, 9), products=(2, 5)),
 Reaction(reactants=(1, 7), products=(2, 3)),
 Reaction(reactants=(3, 9), products=(1, 5)),
 Reaction(reactants=(2, 5), products=(7, 9)),
 Reaction(reactants=(1, 6), products=(0, 7)),
 Reaction(reactants=(1, 5), products=(3, 9)),
 Reaction(reactants=(0, 7), products=(1, 6))]

Pruned partition:


(array([[0, 1, 0, 2, 1, 0, 1, 0],
        [1, 0, 2, 0, 0, 3, 2, 0],
        [1, 1, 1, 0, 1, 0, 0, 2],
        [1, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]),
 Partition(inputs=['S_1', 'S_2', 'S_3', 'S_4'], hidden=['S_6', 'S_7', 'S_8'], output='S_out'))

In [29]:
gen = SyntheticDataGenerator(
    number_of_nonoutput_species=3,
    reaction_rate_ranges=(0.1, 1.0),
    initial_concentration_range=(10, 100),
    noise_level=0.05
)

C, partition, reactions = gen.setup_hardcoded_1latent_example()
print("Composition Matrix:",)
print(C)
print("Null Space of C:")
ns = null_space(C)
print(ns)
print("Reactions:")
for rxn in reactions:
    print(f"{[partition.species[i] for i in rxn.reactants]} → {[partition.species[i] for i in rxn.products]}")
e0 = ns[:,0]
e1 = ns[:,1]
reaction1 = np.array([-1, -1, 1, 0, 0]).T
reaction2 = np.array([0, 0, -1, 1, 1]).T
#solve for reaction vector components of null space basis
a, _, _, _ = lstsq(np.column_stack((e0, e1)), reaction1)
b, _, _, _ = lstsq(np.column_stack((e0, e1)), reaction2)
print(f"Reaction 0 in null space coordinates: {a}")
print(f"Reaction 1 in null space coordinates: {b}")
print(f"Net in null space coordinates: {a + b}")


ns @ a , ns @ b, ns @ (a + b)

Composition Matrix:
[[1 0 1 1 0]
 [0 1 1 1 0]
 [0 1 1 0 1]]
Null Space of C:
[[-0.58293924  0.18756823]
 [-0.58293924  0.18756823]
 [ 0.21178514 -0.67464587]
 [ 0.3711541   0.48707764]
 [ 0.3711541   0.48707764]]
Reactions:
['S_1', 'S_2'] → ['S_3']
['S_3'] → ['S_4', 'S_out']
Reaction 0 in null space coordinates: [ 1.37766362 -1.04978233]
Reaction 1 in null space coordinates: [0.53052307 1.64880116]
Net in null space coordinates: [1.90818669 0.59901883]


(array([-1.00000000e+00, -1.00000000e+00,  1.00000000e+00, -1.21546250e-16,
         1.07880831e-16]),
 array([-3.27704877e-17, -9.16703802e-17, -1.00000000e+00,  1.00000000e+00,
         1.00000000e+00]),
 array([-1.00000000e+00, -1.00000000e+00,  2.47923441e-16,  1.00000000e+00,
         1.00000000e+00]))